In [1]:
import sys
import os

import numpy as nplk
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

sys.path.insert(0, '../../')
import ccal
%matplotlib inline
%config InlineBackend.figure_formats = {'svg',}

==================== Computational Cancer Analysis Library ====================

<21:06:54.947488> Checking dependencies ...
<21:06:54.949794> Using the following packages:
<21:06:54.951510> 	matplotlib (v1.5.1)
<21:06:54.951528> 	numpy (v1.10.4)
<21:06:54.951537> 	pandas (v0.18.0)
<21:06:54.951558> 	rpy2 (v2.7.9)
<21:06:54.951566> 	scikit-learn (v0.17.1)
<21:06:54.951573> 	scipy (v0.17.0)
<21:06:54.951581> 	seaborn (v0.7.0)
